In [1]:
%tensorflow_version 1.x
!apt-get install ffmpeg freeglut3-dev xvfb
!pip install stable-baselines[mpi]==2.10.0

!git clone -l -s git://github.com/wjwswain/chaRL.git chaRL
%cd chaRL/games/blackjack
from env import GameEnv

import gym
import numpy as np
from stable_baselines.deepq.policies import MlpPolicy
from stable_baselines import A2C

TensorFlow 1.x selected.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
freeglut3-dev is already the newest version (2.8.1-3).
freeglut3-dev set to manually installed.
ffmpeg is already the newest version (7:3.4.6-0ubuntu0.18.04.1).
The following package was automatically installed and is no longer required:
  libnvidia-common-440
Use 'apt autoremove' to remove it.
The following NEW packages will be installed:
  xvfb
0 upgraded, 1 newly installed, 0 to remove and 35 not upgraded.
Need to get 784 kB of archives.
After this operation, 2,266 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic-updates/universe amd64 xvfb amd64 2:1.19.6-1ubuntu4.4 [784 kB]
Fetched 784 kB in 1s (925 kB/s)
Selecting previously unselected package xvfb.
(Reading database ... 144465 files and directories currently installed.)
Preparing to unpack .../xvfb_2%3a1.19.6-1ubuntu4.4_amd64.deb ...
Unpacking xvfb (2:1.19.6-1ubuntu4.4) ..

In [ ]:
env = GameEnv()
model = A2C(MlpPolicy, env, verbose=1)
model.learn(total_timesteps=25000)
model.save("dqn_a2c_blackjack")

while True:
    action, _states = model.predict(obs)
    obs, rewards, dones, info = env.step(action)
    env.render()

In [2]:
from stable_baselines.common.env_checker import check_env
env = GameEnv()
check_env(env, warn=True)

AttributeError: ignored